In [77]:
import warnings
warnings.filterwarnings("ignore")

In [78]:
import pandas as pd

In [79]:
import transformers
import torch

In [108]:
torch.cuda.is_available()

True

In [80]:
df = pd.read_csv("data.csv").dropna().reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305954 entries, 0 to 305953
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   input1  305954 non-null  object
 1   input2  305954 non-null  object
 2   label   305954 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 7.0+ MB


In [81]:
df.label.value_counts()

1    152977
0    152977
Name: label, dtype: int64

In [46]:
# df = df.sample(frac=0.5)

In [83]:
test = df.sample(frac=0.2)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61191 entries, 192494 to 73867
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   input1  61191 non-null  object
 1   input2  61191 non-null  object
 2   label   61191 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.9+ MB


In [84]:
test.label.value_counts()

1    30695
0    30496
Name: label, dtype: int64

In [85]:
df = df.drop(test.index).reset_index(drop=True)

In [86]:
from transformers import BertForSequenceClassification, BertTokenizer

model_name = 'bert-base-uncased'
num_labels = 2

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [87]:
sentences = df[['input1', 'input2']].values
labels = df['label'].values

input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent[0],                 # Sentence to encode.
                        sent[1],                 # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 240,          # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])
    
# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will alwa

In [88]:
from sklearn.model_selection import train_test_split

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=44, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=44, test_size=0.2)

In [89]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Create a TensorDataset.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)

# Specify batch size.
batch_size = 64

# Create the DataLoader.
train_dataloader = DataLoader(train_data, 
                              sampler = RandomSampler(train_data), 
                              batch_size = batch_size)

validation_dataloader = DataLoader(validation_data, 
                                   sampler = SequentialSampler(validation_data), 
                                   batch_size = batch_size)

In [91]:
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

# Set the optimizer, learning rate, and number of training epochs.
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8
                )

epochs = 4

# Total number of training steps is [number of batches] x [number of epochs].
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)
                                            
# Set the device to GPU if available.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [92]:
device

device(type='cuda')

In [93]:
from sklearn.metrics import accuracy_score
import numpy as np

def flat_accuracy(preds, labels):
    """
    Calculates the accuracy of a classification model.
    """
    # Convert predictions and labels to numpy arrays
    preds = np.argmax(preds, axis=1).flatten()
    labels = labels.flatten()
    
    # Calculate accuracy using scikit-learn's accuracy_score function
    accuracy = accuracy_score(labels, preds)
    
    return accuracy

In [94]:
# Train the model.
model.to(device)

for epoch in range(epochs):
    print("epoch     ", epoch)
    # Set the model to training mode.
    model.train()

    total_loss = 0

    # Train the model on each batch of the training dataset.
    for step, batch in enumerate(train_dataloader):

        # Unpack this training batch from our dataloader.
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Clear out the gradients.
        model.zero_grad()        

        # Forward pass (evaluate the model on this training batch).
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

        # Get the loss value.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can calculate the average loss at the end. 
        total_loss += loss.item()

        # Backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0 to prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_loss / len(train_dataloader)            

    # Evaluate the model on the validation dataset.
    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        
        # Unpack this validation batch from our dataloader.
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to bother calculating gradients since we're only evaluating the model here.
        with torch.no_grad():

            # Forward pass (evaluate the model on this validation batch).
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask,
                            labels=b_labels)

        # Get the loss value.
        loss = outputs[0]

        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU memory and calculate the accuracy.
        logits = outputs[1]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)

epoch      0
epoch      1
epoch      2
epoch      3


In [107]:
total_eval_loss / len(validation_dataloader)

0.18369388758429622

In [95]:
test_data = test[['input1', 'input2']].values
test_labels = test['label'].values

test_input_ids = []
test_attention_masks = []

for sent in test_data:
    encoded_dict = tokenizer.encode_plus(
                        sent[0],                 # Sentence to encode.
                        sent[1],                 # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 240,          # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    test_input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    test_attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)
test_labels = torch.tensor(test_labels)

test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)

batch_size = 64
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [96]:
from sklearn.metrics import classification_report, f1_score

# Define a function to calculate the flat F1 score
def flat_f1(logits, labels):
    preds = np.argmax(logits, axis=1)
    return f1_score(labels, preds, average='weighted')

In [97]:
total_test_accuracy = 0
total_test_loss = 0
total_test_f1 = 0
total_test_precision = 0
total_test_recall = 0
nb_test_steps = 0

# Set the model to evaluation mode
model.eval()

for batch in test_dataloader:
    
    # Unpack this test batch from our dataloader.
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    # Tell pytorch not to bother calculating gradients since we're only evaluating the model here.
    with torch.no_grad():        

        # Forward pass (evaluate the model on this test batch).
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask,
                        labels=b_labels)

    # Get the loss value.
    loss = outputs[0]

    # Accumulate the test loss.
    total_test_loss += loss.item()

    # Move logits and labels to CPU memory and calculate the accuracy, F1 score, precision, and recall.
    logits = outputs[1]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    total_test_accuracy += flat_accuracy(logits, label_ids)
    total_test_f1 += flat_f1(logits, label_ids)
    metrics = classification_report(label_ids, np.argmax(logits, axis=1), output_dict=True)
    total_test_precision += metrics['weighted avg']['precision']
    total_test_recall += metrics['weighted avg']['recall']
    nb_test_steps += 1
    

In [98]:
nb_test_steps

957

In [99]:
len(test_dataloader)

957

In [100]:
# Calculate the average test loss, accuracy, F1 score, precision, and recall.
avg_test_loss = total_test_loss / len(test_dataloader)
avg_test_accuracy = total_test_accuracy / len(test_dataloader)
avg_test_f1 = total_test_f1 / len(test_dataloader)
avg_test_precision = total_test_precision / len(test_dataloader)
avg_test_recall = total_test_recall / len(test_dataloader)

# Print the evaluation metrics
print("Test Loss: {0:.2f}".format(avg_test_loss))
print("Test Accuracy: {0:.2f}".format(avg_test_accuracy))
print("Test F1 Score: {0:.2f}".format(avg_test_f1))
print("Test Precision: {0:.2f}".format(avg_test_precision))
print("Test Recall: {0:.2f}".format(avg_test_recall))

Test Loss: 0.18
Test Accuracy: 0.95
Test F1 Score: 0.95
Test Precision: 0.96
Test Recall: 0.95


In [101]:
model.save_pretrained("./model")

In [102]:
torch.save(model.state_dict(), 'fine_tuned_bert.pt')

In [103]:
len(test_dataloader)

957

In [104]:
957 * 64

61248

In [105]:
len(test)

61191